실행 위치: Google Colab

# 1. vllm 라이브러리 설치

In [ ]:
!curl -LsSf https://astral.sh/uv/install.sh | sh

In [ ]:
!uv pip install --system vllm==0.11.0 torch==2.8.0 transformers==4.57.6

# 2. 모델 로드
 - 모델 로드 중 실패 발생시 제보 부탁드립니다. (Colab 기본 라이브러리 버전과 충돌 발생 가능성 있음)

In [ ]:
!vllm serve Qwen/Qwen3-1.7B > vllm_server.log 2>&1 &

In [10]:
!cat vllm_server.log

INFO 01-05 11:28:18 [__init__.py:216] Automatically detected platform cuda.
2026-01-05 11:28:19.041318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767612499.062076    1977 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767612499.070922    1977 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767612499.087509    1977 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767612499.087536    1977 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00

# 3. Streaming 요청

## 3-1. curl 명령

In [33]:
!curl http://localhost:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{ \
        "messages": [{"role": "user", "content": "San Francisco is a"}], \
        "max_tokens": 100, \
        "temperature": 0.8, \
        "stream": "true" \
    }'

*   Trying 127.0.0.1:8000...
* Connected to localhost (127.0.0.1) port 8000 (#0)
> POST /v1/chat/completions HTTP/1.1
> Host: localhost:8000
> User-Agent: curl/7.81.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 165
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< date: Mon, 05 Jan 2026 12:13:24 GMT
< server: uvicorn
< content-type: text/event-stream; charset=utf-8
< transfer-encoding: chunked
< 
data: {"id":"chatcmpl-06f9d8c42ac84694a4c489dcde40650b","object":"chat.completion.chunk","created":1767615204,"model":"Qwen/Qwen3-0.6B","choices":[{"index":0,"delta":{"role":"assistant","content":""},"logprobs":null,"finish_reason":null}],"prompt_token_ids":null}

data: {"id":"chatcmpl-06f9d8c42ac84694a4c489dcde40650b","object":"chat.completion.chunk","created":1767615204,"model":"Qwen/Qwen3-0.6B","choices":[{"index":0,"delta":{"content":"<think>"},"logprobs":null,"finish_reason":null,"token_ids":null}]}

data: {"id":"chatcmpl-06f9d8c42ac84694a4c489dcde40650b

## 3-2. requests 라이브러리

In [32]:
import requests, json

with requests.post(
    "http://localhost:8000/v1/chat/completions",
    headers={"Content-Type": "application/json"},
    json={
        "messages": [{"role": "user", "content": "San Francisco is a"}],
        "prompt": "San Francisco is a",
        "stream": True
    }
) as response:
    response.raise_for_status()
    for line in response.iter_lines(decode_unicode=True):
        if line:
            if line.startswith("data: "):
                try:
                    json_data = json.loads(line[len("data: "):])      # "data: " 부분은 제외하고 출력
                    content = json_data.get("choices", [{}])[0].get("delta", {}).get("content", "")
                    if content:
                        print(content)
                except json.JSONDecodeError:
                    continue

<think>


Okay
,
 the
 user
 is
 asking
 "
San
 Francisco
 is
 a
..."
 which
 seems
 to
 be
 a
 bit
 of
 a
 vague
 question
.
 First
,
 I
 need
 to
 figure
 out
 what
 they
're
 really
 asking
.
 Maybe
 they
 want
 to
 know
 what
 San
 Francisco
 is
 in
 general
?
 Or
 perhaps
 they
're
 looking
 for
 information
 on
 a
 specific
 topic
 related
 to
 San
 Francisco
.


Since
 the
 user
 hasn
't
 provided
 more
 context
,
 I
 should
 start
 by
 acknowledging
 the
 ambiguity
.
 I
 can
 explain
 that
 San
 Francisco
 is
 a
 city
 in
 California
,
 known
 for
 its
 history
,
 culture
,
 landmarks
,
 and
 economic
 activity
.
 I
 should
 mention
 key
 points
 like
 its
 role
 as
 a
 major
 city
,
 famous
 attractions
,
 and
 its
 significance
 in
 the
 tech
 industry
.


I
 should
 also
 consider
 if
 there
's
 a
 specific
 area
 they
're
 interested
 in
.
 Maybe
 they
're
 asking
 about
 a
 particular
 event
,
 a
 landmark
,
 or
 a
 notable
 aspect
 of
 San
 Francisco
.
 But
 since
 the
 q

## 3-3. openAI 라이브러리

In [22]:
from openai import OpenAI

model = "Qwen/Qwen3-0.6B"
prompt = "San Francisco is a"
max_tokens = 100
temperature = 0.8

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

stream_resp = client.completions.create(
    model       = model,
    prompt      = prompt,
    max_tokens  = 400,
    temperature = temperature,
    stream      = True
)
for chunk in stream_resp:
    if chunk.choices[0].text:
        print(chunk.choices[0].text, end="")

 city with a population of 632,000. How many people are in each row if the city is divided into 20 rows?
Answer:

To find the number of people in each row, we need to divide the total population by the number of rows. 

Given:
- Total population = 632,000
- Number of rows = 20

We divide 632,000 by 20.

Let me perform the calculation:
632,000 ÷ 20 = 31,600

So, there are 31,600 people in each row.

**Final Answer:** There are 31,600 people in each row.  
$$
\boxed{31600}
$$

Therefore, the answer is $\boxed{31600}$.
**Final Answer**
$$
\boxed{31600}
$$

**Explanation:**
To determine the number of people in each row, we divide the total population by the number of rows. 

$$
\frac{632,000}{20} = 31,600
$$

So, there are 31,600 people in each row.  
$$
\boxed{31600}
$$

**Final Answer**
$$
\boxed{31600}
$$
**Final Answer**
$$
\boxed{31600}
$$
**Final Answer**
$$
\boxed{31600}
$$
**Final Answer**
$$
\boxed{31600}
$$

**Final Answer**
$$
\boxed{31600}
$$
**Final Answer**
$$
\boxed{31600}
$